# Complete Black-Litterman Optimization

| | | | 
|----------------------------------------------------------------------|-----------------------------------------------|---------------------------------------------------------------------------------|
| n=4 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9  | n=4 $returns \sim N(\mu,\sigma)$ Independent  | n=4 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5)  |
| n=8 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9  | n=8 $returns \sim N(\mu,\sigma)$ Independent  | n=8 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5)  |
| n=16 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9 | n=16 $returns \sim N(\mu,\sigma)$ Independent | n=16 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5) |

using the method laid out in 4.2, Black-Litterman portfolio optimization

using https://python-advanced.quantecon.org/black_litterman.html

## Read Packages

In [65]:
import numpy as np
import pandas as pd
import scipy.stats as stat
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
import random
import trace
import pickle
import sklearn.covariance

In [66]:
import pypfopt as pyp
import warnings
from pypfopt.black_litterman import BlackLittermanModel
warnings.filterwarnings("ignore")


## Read Data

In [69]:
## Read the Data
file_name = "PortfolioOptimizationData.pkl"

open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

file_name_os = "PortfolioOptimizationData_Zeros.pkl"

open_file_os = open(file_name_os, "rb")
loaded_list_os = pickle.load(open_file_os)
open_file_os.close()

In [70]:
data_n4_corr = loaded_list[0]
data_n8_corr = loaded_list[1]
data_n16_corr = loaded_list[2]

data_n4_ind = loaded_list[3]
data_n8_ind = loaded_list[4]
data_n16_ind = loaded_list[5]

data_n4_neg = loaded_list[6]
data_n8_neg = loaded_list[7]
data_n16_neg = loaded_list[8]

data_n4_corr_os = loaded_list_os[0]
data_n8_corr_os = loaded_list_os[1]
data_n16_corr_os = loaded_list_os[2]

data_n4_ind_os = loaded_list_os[3]
data_n8_ind_os = loaded_list_os[4]
data_n16_ind_os = loaded_list_os[5]

data_n4_neg_os = loaded_list_os[6]
data_n8_neg_os = loaded_list_os[7]
data_n16_neg_os = loaded_list_os[8]

## Define Functions

Function that Calculated equillibrium values 

This step generates the sample

In [56]:
def bl(n_assets, n_obs, return_vec):
    '''
    This function evaluates the equillibrium returns of a portfolio and generates the sample
    Inputs: 
    n_assets: Number of assets
    n_obs: Number of observations
    return_vec: A matrix of returns of shape n_obs x n_assets (a np.array)
    Returns: 
    weights: optimal weights
    bl_returns: BL returns
    S: BL risk
    '''
    S = pyp.risk_models.CovarianceShrinkage(return_vec).ledoit_wolf()
    view = pd.Series(np.mean(return_vec, axis = 0))
    delta = np.random.normal(1)
    
    bl = pyp.BlackLittermanModel(S, pi = "equal", absolute_views=view)
    bl_return = bl.bl_returns()

    ef = pyp.EfficientFrontier(bl_return, S)
    bl.bl_weights(delta)
    weights = bl.clean_weights()

    S_bl = bl.bl_cov()
    return weights, bl_return, S_bl

In [97]:
def bl_os(n_assets, n_obs, return_vec, return_vec_os):
    '''
    This function evaluates the equillibrium returns of a portfolio and generates the sample
    Inputs: 
    n_assets: Number of assets
    n_obs: Number of observations
    return_vec: A matrix of returns of shape n_obs x n_assets (a np.array)
    Returns: 
    weights: optimal weights
    bl_returns: BL returns
    S: BL risk
    '''
    S = pyp.risk_models.CovarianceShrinkage(return_vec).ledoit_wolf()
    view = pd.Series(np.mean(return_vec_os, axis = 0))
    delta = np.random.normal(1)
    
    bl = pyp.BlackLittermanModel(S, pi = "equal", absolute_views=view)
    bl_return = bl.bl_returns()

    ef = pyp.EfficientFrontier(bl_return, S)
    bl.bl_weights(delta)
    weights = bl.clean_weights()

    S_bl = bl.bl_cov()
    return weights, bl_return, S_bl

Notes: There's no way around it, I have to give it a view, a vector of data, and absolute views. Otherwise all hell breaks loose.

In [58]:
def one_corr_optimization(n_assets, n_obs, data, r):
    '''
    First, simulates portfolios then optimizes. 
    This does 100 replications
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_assets))
    risks_res = np.zeros((99,n_assets))
    
    for i in range(99):
        return_vec = data[i]
        weights, returns, risks = bl(n_assets, n_obs, return_vec) 
        weights = list(weights.items())
        w = [x[1] for x in weights]
        weight_res[i,:] = w
        return_res[i,:] = np.array(returns).T
        risks_res[i,:] = np.diagonal(np.array(risks))
    return weight_res, return_res, risks_res

In [96]:
def one_corr_optimization_zeros(n_assets, n_obs, data, data_os):
    '''
    First, simulates portfolios then optimizes. 
    This does 100 replications
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_assets))
    risks_res = np.zeros((99,n_assets))
    
    for i in range(99):
        return_vec = data[i]
        return_vec_os = data_os[i]
        weights, returns, risks = bl_os(n_assets, n_obs, return_vec, return_vec_os)
        weights = list(weights.items())
        w = [x[1] for x in weights]
        weight_res[i,:] = w
        return_res[i,:] = np.array(returns).T
        risks_res[i,:] = np.diagonal(np.array(risks))
    return weight_res, return_res, risks_res

# Perform Optimization
## Correlated

## $n=4$

In [100]:
n_assets = 4
n_obs = 1000
r = np.random.uniform(0.8, 0.9, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [71]:
n4_weights_corr, n4_returns_corr, n4_risks_corr = one_corr_optimization(n_assets, n_obs, data_n4_corr, r) #

In [61]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_corr, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_corr, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_corr, axis=0),3)))

weights (mean) = [0.266 0.248 0.245 0.241] 
returns (mean) = [3.912 3.96  3.943 3.969] 
risks (mean) =[26.988 27.079 28.098 27.639]


In [ ]:
print('weights (std) = '+str(np.round(np.std(n4_weights_corr, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_corr, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_corr, axis=0),3)))

In [101]:
n4_weights_corr_os, n4_returns_corr_os, n4_risks_corr_os = one_corr_optimization_zeros(n_assets, n_obs, data_n4_corr, data_n4_corr_os)

In [102]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_corr_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_corr_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_corr_os, axis=0),3)))

weights (mean) = [-0.289 -0.298  0.775  0.812] 
returns (mean) = [2.14  2.157 2.797 2.754] 
risks (mean) =[27.495 27.031 28.547 27.613]


In [30]:
print('weights (std) = '+str(np.round(np.std(n4_weights_corr_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_corr_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_corr_os, axis=0),3)))

weights (std) = [0.024 0.023 0.024 0.032] 
returns (std) = [0.008 0.008 0.037 0.039] 
risks (std) =[80363448.239 80362921.541 80363032.365 80392527.224]


## $ n = 8$

In [104]:
n_assets = 8
n_obs = 1000
r = r = np.random.uniform(0.8, 0.9, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [75]:
n8_weights_corr, n8_returns_corr, n8_risks_corr = one_corr_optimization(n_assets, n_obs, data_n8_corr, r) 

In [33]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_corr, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_corr, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_corr, axis=0),3)))

weights (mean) = [0.123 0.124 0.125 0.124 0.126 0.126 0.125 0.126] 
returns (mean) = [4.994 4.994 4.993 4.994 4.995 4.993 4.995 4.993] 
risks (mean) =[71369026.438 71105179.245 71023931.277 71026708.529 71027187.459
 71008524.085 71049610.202 71033945.073]


In [34]:
print('weights (std) = '+str(np.round(np.std(n8_weights_corr, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_corr, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_corr, axis=0),3)))

weights (std) = [0.012 0.012 0.008 0.007 0.005 0.004 0.009 0.005] 
returns (std) = [0.034 0.038 0.034 0.037 0.038 0.037 0.037 0.033] 
risks (std) =[3.35063506e+08 3.35054006e+08 3.35069850e+08 3.35069041e+08
 3.35072102e+08 3.35072748e+08 3.35064570e+08 3.35070701e+08]


In [106]:
n8_weights_corr_os, n8_returns_corr_os, n8_risks_corr_os = one_corr_optimization_zeros(n_assets, n_obs, data_n8_corr, data_n8_corr_os)

In [107]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_corr_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_corr_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_corr_os, axis=0),3)))

weights (mean) = [-0.437 -0.4    0.324  0.301  0.289  0.338  0.3    0.284] 
returns (mean) = [3.125 3.005 3.682 3.719 3.776 3.611 3.757 3.769] 
risks (mean) =[27.621 28.059 26.901 28.186 27.787 27.449 27.351 28.258]


In [37]:
print('weights (std) = '+str(np.round(np.std(n8_weights_corr_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_corr_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_corr_os, axis=0),3)))

weights (std) = [0.01  0.012 0.019 0.01  0.017 0.013 0.012 0.012] 
returns (std) = [0.008 0.007 0.035 0.035 0.036 0.034 0.034 0.035] 
risks (std) =[49382453.037 49444861.627 49466128.58  49385266.678 49999531.526
 49372956.86  49369375.138 49377595.345]


## $n=16$

In [108]:
n_assets = 16
n_obs = 1000
r = np.random.uniform(0.8, 0.9, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [79]:
n16_weights_corr, n16_returns_corr, n16_risks_corr = one_corr_optimization(n_assets, n_obs, data_n16_corr, r) 

In [40]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_corr, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_corr, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_risks_corr, axis=0),3)))

weights (mean) = [0.063 0.062 0.063 0.063 0.062 0.062 0.062 0.062 0.063 0.063 0.062 0.063
 0.063 0.063 0.062 0.062] 
returns (mean) = [5.    4.999 4.999 4.998 4.997 4.999 4.997 5.002 5.002 5.002 4.997 5.001
 5.001 5.001 4.999 4.997] 
risks (mean) =[1.88362128e+08 1.88446506e+08 1.88742708e+08 1.88389878e+08
 1.88498229e+08 1.88433493e+08 1.88834628e+08 1.88512220e+08
 1.88363824e+08 1.88355001e+08 1.88398233e+08 1.88370701e+08
 1.88372286e+08 1.88356901e+08 1.88642166e+08 1.89929768e+08]


In [41]:
print('weights (std) = '+str(np.round(np.std(n16_weights_corr, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_corr, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_risks_corr, axis=0),3)))

weights (std) = [0.002 0.005 0.006 0.003 0.004 0.004 0.008 0.005 0.003 0.003 0.006 0.003
 0.003 0.004 0.006 0.008] 
returns (std) = [0.031 0.033 0.028 0.031 0.03  0.031 0.03  0.031 0.029 0.029 0.032 0.03
 0.033 0.033 0.03  0.031] 
risks (std) =[1.26680051e+09 1.26678829e+09 1.26675136e+09 1.26679650e+09
 1.26678167e+09 1.26679037e+09 1.26674065e+09 1.26677997e+09
 1.26680028e+09 1.26680152e+09 1.26679521e+09 1.26679927e+09
 1.26679907e+09 1.26680124e+09 1.26676306e+09 1.26667478e+09]


In [109]:
n16_weights_corr_os, n16_returns_corr_os, n16_risks_corr_os = one_corr_optimization_zeros(n_assets, n_obs, data_n16_corr, data_n16_corr_os)

In [110]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_corr_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_corr_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_risks_corr_os, axis=0),3)))

weights (mean) = [-0.468 -0.468  0.145  0.145  0.114  0.134  0.135  0.153  0.148  0.138
  0.138  0.122  0.149  0.123  0.149  0.145] 
returns (mean) = [3.615 3.62  4.249 4.26  4.404 4.308 4.332 4.222 4.221 4.277 4.295 4.38
 4.253 4.373 4.231 4.263] 
risks (mean) =[27.727 27.832 28.681 28.294 29.462 29.16  30.101 28.64  28.967 29.336
 28.839 29.522 28.278 28.918 29.386 29.174]


In [44]:
print('weights (std) = '+str(np.round(np.std(n16_weights_corr_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_corr_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_risks_corr_os, axis=0),3)))

weights (std) = [0.003 0.003 0.002 0.002 0.004 0.005 0.002 0.002 0.004 0.001 0.004 0.002
 0.002 0.002 0.005 0.006] 
returns (std) = [0.008 0.007 0.032 0.032 0.033 0.032 0.029 0.033 0.034 0.036 0.035 0.031
 0.032 0.035 0.033 0.032] 
risks (std) =[1.06840197e+10 1.06839889e+10 1.06840200e+10 1.06840135e+10
 1.06839523e+10 1.06839996e+10 1.06840214e+10 1.06840221e+10
 1.06840188e+10 1.06840220e+10 1.06840155e+10 1.06840203e+10
 1.06840199e+10 1.06840221e+10 1.06840139e+10 1.06840013e+10]


## Independent

In [111]:
n_assets = 4
n_obs = 1000
r = np.eye(n_assets)

In [82]:
n4_weights_ind, n4_returns_ind, n4_risks_ind = one_corr_optimization(n_assets, n_obs, data_n4_ind, r) 

In [47]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_ind, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_ind, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_ind, axis=0),3)))

weights (mean) = [0.252 0.246 0.25  0.252] 
returns (mean) = [5.003 5.    4.997 4.998] 
risks (mean) =[41366410.454 41420949.132 41374162.93  41368818.21 ]


In [48]:
print('weights (std) = '+str(np.round(np.std(n4_weights_ind, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_ind, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_ind, axis=0),3)))

weights (std) = [0.014 0.022 0.017 0.009] 
returns (std) = [0.036 0.03  0.035 0.032] 
risks (std) =[2.25041714e+08 2.25032533e+08 2.25040306e+08 2.25041286e+08]


In [112]:
n4_weights_ind_os, n4_returns_ind_os, n4_risks_ind_os = one_corr_optimization_zeros(n_assets, n_obs, data_n4_ind, data_n4_ind_os)

In [113]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_ind_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_ind_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_ind_os, axis=0),3)))

weights (mean) = [0.111 0.111 0.388 0.39 ] 
returns (mean) = [0.745 0.749 2.625 2.628] 
risks (mean) =[27.901 27.708 27.862 27.726]


In [114]:
print('weights (std) = '+str(np.round(np.std(n4_weights_ind_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_ind_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_ind_os, axis=0),3)))

weights (std) = [0.004 0.004 0.007 0.007] 
returns (std) = [0.018 0.015 0.019 0.02 ] 
risks (std) =[1.716 1.736 1.711 1.694]


## $n=8$

In [115]:
n_assets = 8
n_obs = 1000
r = np.eye(n_assets)

In [116]:
n8_weights_ind, n8_returns_ind, n8_risks_ind = one_corr_optimization(n_assets, n_obs, data_n8_ind, r) 

In [88]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_ind, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_ind, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_ind, axis=0),3)))

weights (mean) = [0.126 0.125 0.125 0.125 0.124 0.125 0.125 0.125] 
returns (mean) = [2.554 2.562 2.558 2.565 2.564 2.56  2.561 2.56 ] 
risks (mean) =[27.665 27.821 27.774 27.77  27.881 27.841 27.667 27.65 ]


In [89]:
print('weights (std) = '+str(np.round(np.std(n8_weights_ind, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_ind, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_ind, axis=0),3)))

weights (std) = [0.004 0.004 0.003 0.003 0.003 0.004 0.003 0.003] 
returns (std) = [0.042 0.034 0.038 0.033 0.033 0.031 0.031 0.028] 
risks (std) =[1.268 1.302 1.185 1.232 1.291 1.568 1.216 1.279]


In [117]:
n8_weights_ind_os, n8_returns_ind_os, n8_risks_ind_os = one_corr_optimization_zeros(n_assets, n_obs, data_n8_ind, data_n8_ind_os)

In [118]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_ind_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_ind_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_ind_os, axis=0),3)))

weights (mean) = [0.042 0.041 0.153 0.153 0.152 0.153 0.154 0.153] 
returns (mean) = [0.681 0.691 2.56  2.562 2.562 2.563 2.563 2.56 ] 
risks (mean) =[27.665 27.821 27.774 27.77  27.881 27.841 27.667 27.65 ]


In [58]:
print('weights (std) = '+str(np.round(np.std(n8_weights_ind_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_ind_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_ind_os, axis=0),3)))

weights (std) = [0.016 0.006 0.007 0.009 0.013 0.012 0.019 0.014] 
returns (std) = [0.008 0.008 0.034 0.036 0.033 0.032 0.034 0.038] 
risks (std) =[4.58954499e+08 4.59027845e+08 4.59020864e+08 4.59027301e+08
 4.58980185e+08 4.59010077e+08 4.58999154e+08 4.58943942e+08]


## $n=16$

In [119]:
n_assets = 16
n_obs = 1000
r = np.eye(n_assets)

In [120]:
n16_weights_ind, n16_returns_ind, n16_risks_ind = one_corr_optimization(n_assets, n_obs, data_n16_ind, r) 

In [121]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_ind, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_ind, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_risks_ind, axis=0),3)))

weights (mean) = [0.063 0.062 0.062 0.063 0.063 0.062 0.063 0.062 0.062 0.063 0.062 0.062
 0.063 0.062 0.063 0.062] 
returns (mean) = [2.526 2.53  2.529 2.533 2.536 2.531 2.53  2.526 2.526 2.533 2.53  2.526
 2.521 2.532 2.539 2.536] 
risks (mean) =[31.632 31.578 31.66  31.551 31.45  31.733 31.539 31.733 31.648 31.591
 31.584 31.69  31.573 31.625 31.473 31.634]


In [64]:
print('weights (std) = '+str(np.round(np.std(n16_weights_ind, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_ind, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_risks_ind, axis=0),3)))

weights (std) = [0.003 0.004 0.006 0.004 0.003 0.005 0.004 0.004 0.004 0.008 0.007 0.005
 0.004 0.004 0.006 0.003] 
returns (std) = [0.034 0.038 0.032 0.033 0.033 0.031 0.036 0.031 0.032 0.03  0.036 0.037
 0.032 0.035 0.032 0.034] 
risks (std) =[3.21144975e+08 3.21131210e+08 3.23845349e+08 3.21144501e+08
 3.21138239e+08 3.21148696e+08 3.21137789e+08 3.21132460e+08
 3.21143971e+08 3.21080466e+08 3.21085888e+08 3.21142429e+08
 3.21133494e+08 3.21392719e+08 3.21133735e+08 3.21142265e+08]


In [142]:
n16_weights_ind_os, n16_returns_ind_os, n16_risks_ind_os = one_corr_optimization_zeros(n_assets, n_obs, data_n16_ind, data_n16_ind_os)

In [123]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_ind_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_ind_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_ind_risks_os, axis=0),3)))

weights (mean) = [0.018 0.018 0.069 0.069 0.069 0.069 0.069 0.069 0.069 0.069 0.069 0.069
 0.069 0.069 0.069 0.069] 
returns (mean) = [0.649 0.657 2.529 2.532 2.534 2.534 2.53  2.529 2.529 2.532 2.529 2.527
 2.52  2.53  2.534 2.534] 
risks (mean) =[31.632 31.578 31.66  31.551 31.45  31.733 31.539 31.733 31.648 31.591
 31.584 31.69  31.573 31.625 31.473 31.634]


In [69]:
print('weights (std) = '+str(np.round(np.std(n16_weights_ind_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_ind_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_ind_risks_os, axis=0),3)))

weights (std) = [0.008 0.003 0.002 0.002 0.002 0.003 0.002 0.002 0.003 0.002 0.004 0.006
 0.002 0.004 0.006 0.002] 
returns (std) = [0.007 0.008 0.03  0.037 0.034 0.037 0.035 0.036 0.031 0.036 0.033 0.033
 0.036 0.034 0.033 0.032] 
risks (std) =[3.92165310e+08 3.92212204e+08 3.92217260e+08 3.92212640e+08
 3.92216153e+08 3.92211557e+08 3.92216772e+08 3.92215951e+08
 3.92205642e+08 3.92214752e+08 3.92209495e+08 3.92183795e+08
 3.92215626e+08 3.92213928e+08 3.92183642e+08 3.92216588e+08]


## Negatively Correlated

## $n=4$

In [124]:
n_assets = 4
n_obs = 1000
r = r = np.random.uniform(-0.8, -0.5, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [125]:
n4_weights_neg, n4_returns_neg, n4_risks_neg = one_corr_optimization(n_assets, n_obs, data_n4_neg, r) 

In [126]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_neg, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_neg, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_neg, axis=0),3)))

weights (mean) = [0.25  0.237 0.257 0.256] 
returns (mean) = [2.477 2.461 2.469 2.474] 
risks (mean) =[140.732 143.802 139.421 139.73 ]


In [127]:
print('weights (std) = '+str(np.round(np.std(n4_weights_neg, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_neg, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_neg, axis=0),3)))

weights (std) = [0.015 0.015 0.015 0.014] 
returns (std) = [0.136 0.139 0.136 0.136] 
risks (std) =[359.293 358.531 359.621 359.537]


In [128]:
n4_weights_neg_os, n4_returns_neg_os, n4_risks_neg_os = one_corr_optimization_zeros(n_assets, n_obs, data_n4_neg, data_n4_neg_os)

In [129]:
print('weights (mean) = '+str(np.round(np.mean(n4_weights_neg_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_returns_neg_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_risks_neg_os, axis=0),3)))

weights (mean) = [0.121 0.115 0.383 0.381] 
returns (mean) = [0.642 0.632 2.547 2.546] 
risks (mean) =[140.732 143.802 139.421 139.73 ]


In [82]:
print('weights (std) = '+str(np.round(np.std(n4_weights_neg_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_returns_neg_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_risks_neg_os, axis=0),3)))

weights (std) = [0.035 0.027 0.02  0.022] 
returns (std) = [0.011 0.01  0.045 0.042] 
risks (std) =[3.55828886e+09 3.55829325e+09 3.55829551e+09 3.55829474e+09]


## $n=8$

In [130]:
n_assets = 8
n_obs = 1000
r = r = np.random.uniform(-0.8, -0.5, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [131]:
n8_weights_neg, n8_returns_neg, n8_risks_neg = one_corr_optimization(n_assets, n_obs, data_n8_neg, r) 

In [132]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_neg, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_neg, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_neg, axis=0),3)))

weights (mean) = [0.128 0.122 0.126 0.121 0.122 0.127 0.127 0.127] 
returns (mean) = [2.617 2.641 2.62  2.633 2.626 2.618 2.623 2.617] 
risks (mean) =[65811.683 65816.267 65813.675 65822.176 65817.51  65812.45  65813.023
 65819.143]


In [88]:
print('weights (std) = '+str(np.round(np.std(n8_weights_neg, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_neg, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_neg, axis=0),3)))

weights (std) = [0.011 0.01  0.01  0.013 0.016 0.018 0.013 0.009] 
returns (std) = [0.048 0.046 0.049 0.046 0.052 0.048 0.044 0.045] 
risks (std) =[1.21819660e+09 1.21820042e+09 1.21819552e+09 1.21816814e+09
 1.21817365e+09 1.21815383e+09 1.21819658e+09 1.21819837e+09]


In [133]:
n8_weights_neg_os, n8_returns_neg_os, n8_risks_neg_os = one_corr_optimization_zeros(n_assets, n_obs, data_n8_neg, data_n8_neg_os)

In [90]:
print('weights (mean) = '+str(np.round(np.mean(n8_weights_neg_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_returns_neg_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_risks_neg_os, axis=0),3)))

weights (mean) = [0.126 0.125 0.122 0.124 0.125 0.125 0.126 0.127] 
returns (mean) = [5.    5.    4.999 5.003 5.005 4.996 5.002 5.003] 
risks (mean) =[43475363.595 43451051.369 43539793.923 43494707.403 43549804.317
 43463337.628 43493135.641 43454228.897]


In [91]:
print('weights (std) = '+str(np.round(np.std(n8_weights_neg_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_returns_neg_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_risks_neg_os, axis=0),3)))

weights (std) = [0.012 0.009 0.015 0.013 0.013 0.009 0.011 0.009] 
returns (std) = [0.011 0.01  0.05  0.041 0.047 0.052 0.045 0.049] 
risks (std) =[1.57288003e+08 1.57294193e+08 1.57272035e+08 1.57285533e+08
 1.57271436e+08 1.57290992e+08 1.57284416e+08 1.57296817e+08]


## $n=16$

In [135]:
n_assets = 16
n_obs = 1000
r = r = np.random.uniform(-0.8, -0.5, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [136]:
n16_weights_neg, n16_returns_neg, n16_risks_neg = one_corr_optimization(n_assets, n_obs, data_n16_neg, r) 

In [94]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_neg, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_neg, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_risks_neg, axis=0),3)))

weights (mean) = [0.063 0.063 0.063 0.062 0.063 0.062 0.063 0.062 0.063 0.063 0.061 0.063
 0.062 0.063 0.062 0.063] 
returns (mean) = [5.004 5.004 5.002 4.993 5.001 5.001 4.991 4.998 5.004 4.992 4.992 4.998
 5.003 4.993 5.002 4.993] 
risks (mean) =[89022720.013 88822370.167 88954223.816 88767055.623 88750580.809
 88762651.569 88750199.795 88742435.26  88738380.137 88754234.1
 88869480.71  88737452.771 88762099.267 88734509.496 88752760.405
 88749680.522]


In [95]:
print('weights (std) = '+str(np.round(np.std(n16_weights_neg, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_neg, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_risks_neg, axis=0),3)))

weights (std) = [0.006 0.003 0.006 0.003 0.003 0.005 0.002 0.003 0.002 0.003 0.007 0.003
 0.007 0.002 0.005 0.004] 
returns (std) = [0.056 0.046 0.05  0.043 0.05  0.049 0.047 0.046 0.045 0.055 0.049 0.054
 0.051 0.047 0.052 0.05 ] 
risks (std) =[4.77522527e+08 4.77553159e+08 4.77531856e+08 4.77558289e+08
 4.77561163e+08 4.77559019e+08 4.77561269e+08 4.77562517e+08
 4.77563552e+08 4.77560435e+08 4.77541238e+08 4.77563427e+08
 4.77558907e+08 4.77563960e+08 4.77560730e+08 4.77561226e+08]


In [137]:
n16_weights_neg_os, n16_returns_neg_os, n16_risks_neg_os = one_corr_optimization_zeros(n_assets, n_obs, data_n16_neg, data_n16_neg_os)

In [138]:
print('weights (mean) = '+str(np.round(np.mean(n16_weights_neg_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_returns_neg_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_risks_neg_os, axis=0),3)))

weights (mean) = [0.017 0.017 0.069 0.069 0.069 0.068 0.069 0.07  0.07  0.069 0.069 0.069
 0.068 0.07  0.069 0.069] 
returns (mean) = [0.687 0.689 2.562 2.556 2.553 2.555 2.558 2.553 2.559 2.559 2.562 2.554
 2.554 2.558 2.55  2.553] 
risks (mean) =[52564.735 52680.915 52693.577 52599.713 52569.793 52601.014 52562.357
 52558.735 52559.373 52572.397 52561.921 52575.614 52627.783 52559.538
 52568.25  52578.751]


In [98]:
print('weights (std) = '+str(np.round(np.std(n16_weights_neg_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_returns_neg_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_risks_neg_os, axis=0),3)))

weights (std) = [0.005 0.002 0.003 0.006 0.005 0.002 0.003 0.006 0.006 0.005 0.007 0.007
 0.002 0.005 0.003 0.005] 
returns (std) = [0.012 0.01  0.046 0.044 0.044 0.048 0.05  0.055 0.048 0.051 0.052 0.054
 0.05  0.051 0.053 0.046] 
risks (std) =[1.09402416e+10 1.09403576e+10 1.09403556e+10 1.09403437e+10
 1.09403483e+10 1.09403563e+10 1.09403523e+10 1.09401958e+10
 1.09403286e+10 1.09403256e+10 1.09403255e+10 1.09398427e+10
 1.09403567e+10 1.09402520e+10 1.09403562e+10 1.09403266e+10]


## Save the Results

In [139]:
BlackLittermanResults_Weights = [n4_weights_corr, n8_weights_corr, n16_weights_corr, n4_weights_corr_os, 
                              n8_weights_corr_os, n16_weights_corr_os, n4_weights_ind, n8_weights_ind,
                              n16_weights_ind, n4_weights_ind_os, n8_weights_ind_os, n16_weights_ind_os,
                              n4_weights_neg, n8_weights_neg, n16_weights_neg, n4_weights_neg_os, 
                              n8_weights_neg_os, n16_weights_neg_os]

file_name = "BlackLittermanResultsWeights.pkl"
open_file = open(file_name, "wb")
pickle.dump(BlackLittermanResults_Weights, open_file)
open_file.close()

In [140]:
BlackLittermanResults_Returns = [n4_returns_corr, n8_returns_corr, n16_returns_corr, n4_returns_corr_os, 
                              n8_returns_corr_os, n16_returns_corr_os, n4_returns_ind, n8_returns_ind,
                              n16_returns_ind, n4_returns_ind_os, n8_returns_ind_os, n16_returns_ind_os,
                              n4_returns_neg, n8_returns_neg, n16_returns_neg, n4_returns_neg_os, 
                              n8_returns_neg_os, n16_returns_neg_os]

file_name = "BlackLittermanResultsReturns.pkl"
open_file = open(file_name, "wb")
pickle.dump(BlackLittermanResults_Returns, open_file)
open_file.close()

In [143]:
BlackLittermanResults_Risks = [n4_risks_corr, n8_risks_corr, n16_risks_corr, n4_risks_corr_os, 
                              n8_risks_corr_os, n16_risks_corr_os, n4_risks_ind, n8_risks_ind,
                              n16_risks_ind, n4_risks_ind_os, n8_risks_ind_os, n16_risks_ind_os,
                              n4_risks_neg, n8_risks_neg, n16_risks_neg, n4_risks_neg_os, 
                              n8_risks_neg_os, n16_risks_neg_os]

file_name = "BlackLittermanResultsRisks.pkl"
open_file = open(file_name, "wb")
pickle.dump(BlackLittermanResults_Risks, open_file)
open_file.close()

## Play with the view's values

I let the optimization run with 100 repetitions. In each repetition I randomly select an amount between -10% and 10% of the default views. Then I select the maximum returns and found that returns are maximized when views are 9.9% higher than the prior value.

So, The higher the views, the higher the calculated returns. 
So... I'll just use the views equal to the prior so that views don't sway the results. right.?

In [1]:
def bl_views(n_assets, n_obs, return_vec, views):
    '''
    This function evaluates the equillibrium returns of a portfolio and generates the sample
    Inputs: 
    n_assets: Number of assets
    n_obs: Number of observations
    return_vec: A matrix of returns of shape n_obs x n_assets (a np.array)
    Returns: 
    weights: optimal weights
    bl_returns: BL returns
    S: BL risk
    '''
    market_prices = pd.Series(np.random.randn(n_assets)) 
    rng = np.random.default_rng()
    cov_struct = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs)
    S = pyp.risk_models.CovarianceShrinkage(cov_struct).ledoit_wolf()
    delta = pyp.black_litterman.market_implied_risk_aversion(market_prices, risk_free_rate=0.5)
    
    market_prior = pd.Series(np.mean(return_vec + 5, axis = 0))
    view = pd.Series(views)#np.ones(n_assets)*(1/n_assets)+ 5.0)
    
    bl = pyp.BlackLittermanModel(S, pi=market_prior, absolute_views=view)
    bl_return = bl.bl_returns()

    ef = pyp.EfficientFrontier(bl_return, r)
    bl.bl_weights(delta)
    weights = bl.clean_weights()

    S_bl = bl.bl_cov()
    return weights, bl_return, S_bl

In [118]:
def views_optimization(n_assets, n_obs, r):
    '''
    First, simulates portfolios then optimizes. 
    This does 100 replications
    '''
    weight_diff = np.zeros((99,n_assets))
    return_diff = np.zeros((99,n_assets))
    risks_diff = np.zeros((99,n_assets))
    view_diff = np.zeros((99,n_assets))
    
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_assets))
    risks_res = np.zeros((99,n_assets))
    views = np.zeros((99,n_assets))
    
    rng = np.random.default_rng()
    return_vec = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs)
    
    for i in range(99):
        # Make default calculation
        default = np.mean(return_vec + 5, axis = 0)
        default_weights, default_ret, default_rsk = bl_views(n_assets, n_obs, return_vec, default)
        default_weights = list(default_weights.items())
        wt = [x[1] for x in default_weights]
        weight_def = wt
        return_def = np.array(default_ret).T
        risks_def = np.diagonal(np.array(default_rsk))
        
        views[i] = np.mean(return_vec + 5, axis = 0) + np.round(np.random.uniform(-0.1,0.1,1),3) * np.mean(return_vec + 5, axis=0)
        weights, returns, risks = bl_views(n_assets, n_obs, return_vec, views[i]) 
        weights = list(weights.items())
        w = [x[1] for x in weights]
        weight_res[i,:] = w
        return_res[i,:] = np.array(returns).T
        risks_res[i,:] = np.diagonal(np.array(risks))
        view_diff[i] = (views[i]-default)/default * 100
        weight_diff[i,:] = (weight_res[i] - weight_def)/weight_def * 100
        return_diff[i,:] = (return_res[i] - return_def)/return_def * 100
        risks_diff[i,:] = (risks_res[i] - risks_def)/risks_def * 100
    return views, view_diff, weight_diff, return_res, risks_diff

In [28]:
n_assets = 4
n_obs = 1000
r = np.random.uniform(0.8, 0.9, (n_assets,n_assets))
np.fill_diagonal(r, 1)

In [119]:
views, views_diff, views_weights, views_returns, views_risks = views_optimization(n_assets, n_obs, r) 

In [95]:
out = pd.DataFrame(np.hstack((views_diff, views_returns)))

In [120]:
views_diff[views_returns.sum(axis=1).argmax()]
#np.where(x == np.max(views_returns, axis=0))
#views_returns.sum(axis=1)#.argmax()

array([9.9, 9.9, 9.9, 9.9])

In [105]:
type(views_returns[1,1])

numpy.float64

In [113]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(out)

       0     1     2     3         4         5         6         7
0    3.8   3.8   3.8   3.8  5.052776  5.044410  5.079386  5.068058
1   -1.5  -1.5  -1.5  -1.5  4.921375  4.913226  4.947292  4.936259
2   -9.4  -9.4  -9.4  -9.4  4.725995  4.717771  4.750990  4.739886
3    2.7   2.7   2.7   2.7  5.025504  5.017183  5.051971  5.040703
4   11.8  11.8  11.8  11.8  5.251559  5.244740  5.279746  5.267524
5   14.6  14.6  14.6  14.6  5.320539  5.311729  5.348559  5.336630
6   18.1  18.1  18.1  18.1  5.407314  5.398360  5.435791  5.423667
7   18.9  18.9  18.9  18.9  5.427148  5.418162  5.455729  5.443562
8    3.9   3.9   3.9   3.9  5.055256  5.046885  5.081879  5.070545
9    5.2   5.2   5.2   5.2  5.087486  5.079062  5.114279  5.102873
10  15.7  15.7  15.7  15.7  5.347777  5.339086  5.376023  5.363993
11  17.5  17.5  17.5  17.5  5.392438  5.383509  5.420837  5.408747
12  10.6  10.6  10.6  10.6  5.221368  5.212722  5.248865  5.237159
13  15.1  15.1  15.1  15.1  5.332926  5.324098  5.361025  5.34

Archived:
    def bl(n_assets, n_obs, return_vec):
    '''
    This function evaluates the equillibrium returns of a portfolio and generates the sample
    Inputs: 
    n_assets: Number of assets
    n_obs: Number of observations
    return_vec: A matrix of returns of shape n_obs x n_assets (a np.array)
    Returns: 
    weights: optimal weights
    bl_returns: BL returns
    S: BL risk
    '''
    market_prices = pd.Series(np.random.randn(n_assets)) # This needs to be Rf - Rm1
    cov_struct = return_vec
    S = pyp.risk_models.CovarianceShrinkage(cov_struct).ledoit_wolf()
    delta = pyp.black_litterman.market_implied_risk_aversion(market_prices, risk_free_rate=0.5)
    
    data = pd.Series(np.mean(return_vec + 5, axis = 0))
    view = pd.Series(np.mean(return_vec + 5, axis = 0))#np.ones(n_assets)*(1/n_assets)+ 5.0)
    
    bl = pyp.BlackLittermanModel(S, pi=data, absolute_views=view)
    bl_return = bl.bl_returns()

    ef = pyp.EfficientFrontier(bl_return, r)
    bl.bl_weights(delta)
    weights = bl.clean_weights()

    S_bl = bl.bl_cov()
    return weights, bl_return, S_bl
    
    ### This is a test - do not run
#r = np.random.uniform(0.8, 0.9, (4,4))
#rng = np.random.default_rng()
#return_vec = rng.multivariate_normal(np.zeros(4), cov = r, size = 100)


#market_prices = pd.Series(np.mean(return_vec)) 
#rng = np.random.default_rng()
#cov_struct = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs)
#S = pyp.risk_models.CovarianceShrinkage(cov_struct).ledoit_wolf()
#delta = pyp.black_litterman.market_implied_risk_aversion(market_prices, risk_free_rate=0.5)
    
#market_prior = pd.Series(np.random.randn(n_assets))
#view = pd.Series(np.ones(n_assets)*(1/n_assets))
    
#bl = pyp.BlackLittermanModel(S, pi=market_prior, absolute_views=view)
#bl_return = bl.bl_returns()

#ef = pyp.EfficientFrontier(bl_return, r)
#bl.bl_weights(delta)
#weights = bl.clean_weights()

#S_bl = bl.bl_cov()
#print(cov_struct, S, S_bl)
#np.mean(return_vec, axis = 0)

In [ ]:
def one_corr_optimization_zeros(n_assets, n_obs, data, r):
    '''
    First, simulates portfolios then optimizes. 
    This does 100 replications
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_assets))
    risks_res = np.zeros((99,n_assets))
    
    for i in range(99):
        return_vec = data[i]
        _75_perct = int(len(return_vec[:,0])*3/4)
        return_vec[random.sample(list(range(1000)), _75_perct),0] = 0
        np.random.seed(i*5)
        return_vec[random.sample(list(range(1000)), _75_perct),1] = 0
        
        weights, returns, risks = bl(n_assets, n_obs, return_vec)
        weights = list(weights.items())
        w = [x[1] for x in weights]
        weight_res[i,:] = w
        return_res[i,:] = np.array(returns).T
        risks_res[i,:] = np.diagonal(np.array(risks))
    return weight_res, return_res, risks_res